**Sistema de Aprendizado Interativo com Agentes**

Este notebook implementa um sistema de aprendizado interativo utilizando múltiplos agentes para guiar o usuário no estudo de um tópico, incluindo busca de informação, redação de aula,material de revisão, questionário e sugestões de aprofundamento, com opção de salvar o histórico localmente.

Desenvolvido com base na Imersão IA Alura + Google e o Google Agent Development Kit (ADK).

Sugestão de Melhoria Futura: Integrar com Google Docs ou Google Drive para salvar documentos online.

Feito por: Juliana Hagen, com a ajudo do Google Gemini - Assistente Python

In [ ]:
# @title Instalações e Importações Necessárias
# Instalar bibliotecas
%pip -q install google-genai google-adk python-dotenv

# Importar bibliotecas
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search # Importação correta da ferramenta de busca (lowercase)
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown, clear_output
import requests # Necessário para verificar links
import warnings
import re # Necessário para extrair URLs
import time # Pode ser útil para delays (opcional)

# Ignorar avisos (opcional, útil para manter a saída limpa)
warnings.filterwarnings("ignore")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
# @title Configuração da API Key e Cliente Gemini
# Configura a API Key do Google Gemini
# É recomendado salvar sua API Key como um "Secret" no Google Colab
# Vá no ícone de chave (🔑) na barra lateral esquerda, clique em "+ New secret"
# Crie um Secret chamado GOOGLE_API_KEY e cole sua chave lá.
# Se você estiver rodando fora do Colab, pode precisar usar variáveis de ambiente ou um arquivo .env
try:
    # Tenta obter a API Key dos secrets do Colab
    API_KEY = userdata.get('GOOGLE_API_KEY')
    if not API_KEY:
        # Se não encontrar nos secrets, tenta de variáveis de ambiente (para rodar localmente, por exemplo)
        API_KEY = os.environ.get("GOOGLE_API_KEY")
        if not API_KEY:
             raise ValueError("API Key não encontrada nos secrets do Colab ou variáveis de ambiente.")
    os.environ["GOOGLE_API_KEY"] = API_KEY # Define a variável de ambiente que o SDK do GenAI procura
    print("✅ API Key configurada.")
except userdata.NotebookAccessError:
    print("Por favor, habilite o 'Notebook access' para ler secrets no Colab (ícone de chave na barra lateral).")
except ValueError as e:
    print(f"Erro ao obter a API Key: {e}")
    print("Certifique-se de ter salvo sua chave como um Secret chamado GOOGLE_API_KEY no Colab.")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao configurar a API Key: {e}")

✅ API Key configurada.


In [ ]:
# Configura o cliente da SDK do Gemini
try:
    client = genai.Client()
    MODEL_ID = "gemini-2.0-flash" # Modelo a ser usado pelos agentes por padrão. Pode ser alterado.
    print("✅ Cliente Gemini configurado com sucesso.")
except Exception as e:
    print(f"❌ Erro ao configurar o cliente Gemini: {e}")
    print("Verifique sua API Key e sua conexão com a internet.")

✅ Cliente Gemini configurado com sucesso.


In [ ]:
# @title Funções Auxiliares (call_agent, to_markdown, verificar links)

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
# Esta função foi revisada para garantir que a instância de Runner é criada corretamente dentro dela.
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória. Cada chamada é uma nova sessão por padrão aqui.
    # Para interações multi-turno contínuas com o MESMO agente (como no quiz),
    # uma única instância de session_service e session DEVE ser mantida e reutilizada
    # nas chamadas subsequentes para o mesmo agente. A implementação abaixo no fluxo principal
    # para o quiz tenta simular isso passando todo o histórico na entrada a cada turno,
    # já que call_agent inicia uma nova sessão sempre. Uma implementação mais robusta
    # do quiz usaria diretamente a classe Runner fora desta função call_agent.
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria a instância de Runner para este agente e sessão. ESTA LINHA É CRUCIAL.
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        # Itera pelos eventos retornados durante a execução do agente
        # Em um ambiente assíncrono real, usaria 'async for' e 'await'
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
        return final_response.strip()
    except Exception as e:
        print(f"❌ Erro durante a execução do agente {agent.name}: {e}")
        # Imprimir o traceback completo para depuração pode ser útil durante o desenvolvimento
        # import traceback
        # traceback.print_exc()
        return f"Desculpe, o agente {agent.name} encontrou um erro ao processar sua solicitação."


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    # Substitui marcadores de lista comuns por marcadores Markdown para melhor renderização
    # Substitui o espaço antes do '*' para 4 espaços, padrão para listas aninhadas em Markdown
    text = text.replace('•', '    *')
    # Retorna o texto como objeto Markdown para o Colab interpretar
    return Markdown(text)

# Função auxiliar para extrair URLs do texto
def extract_urls(text):
    # Expressão regular para encontrar URLs que começam com http ou https
    # Adapte se o formato de saída do agente variar muito (ex: links sem http/https)
    url_pattern = re.compile(r'https?://[^\s\'"]+')
    return url_pattern.findall(text)

# Função auxiliar para verificar se uma URL está ativa
def is_url_active(url, timeout=5):
    try:
        # Tenta fazer uma requisição HEAD primeiro (mais rápida, só pede o cabeçalho)
        # Permite redirecionamentos (allow_redirects=True por padrão para HEAD)
        response = requests.head(url, timeout=timeout, allow_redirects=True)
        # Verifica se o status code indica sucesso (2xx)
        response.raise_for_status() # Lança exceções para códigos de erro (4xx, 5xx)
        return True
    except requests.exceptions.RequestException:
        # Captura todos os erros de requisição (timeout, conexão, http errors como 404, 500)
        return False
    except Exception as e:
        # Captura outros erros inesperados
        # print(f"Erro inesperado ao verificar URL {url}: {e}") # Opcional para depuração
        return False


In [ ]:
# @title Definição dos Agentes (Instruções Ajustadas)

# --- Agente 1: Agente Recepcionista ---
# Função para criar e configurar o Agente Recepcionista
def criar_agente_recepcionista():
    print(f"Configurando o Agente Recepcionista...")
    agente = Agent(
        name="agente_recepcionista",
        model=MODEL_ID,
        instruction="""
        Você é um professor atencioso que vai preparar uma aula especial para um aluno.
        Seu objetivo principal é entender o perfil do aluno para adaptar o conteúdo.

        Você receberá uma MENSAGEM DE ENTRADA que conterá o TÓPICO e a DATA de hoje, possivelmente seguido por respostas do aluno.
        Formato esperado da entrada inicial:
        TÓPICO: [Assunto]
        DATA: [Data]
        ---
        [Instrução para iniciar]

        Formato esperado da entrada com respostas:
        Respostas do Aluno:
        Nível: [resposta]
        Público/Complexidade: [resposta]
        Estilo: [resposta]
        ---
        [Instrução para processar]

        Na sua PRIMEIRA interação (quando receber o TÓPICO e a DATA):
        1. Extraia o TÓPICO e a DATA da mensagem de entrada.
        2. Apresente-se brevemente como o professor que preparará a aula.
        3. Faça AS SEGUINTES perguntas ao aluno sobre o TÓPICO para coletar o perfil dele:
           - Qual é o seu nível de conhecimento prévio sobre o tópico? (Sugira opções como nenhum, básico, intermediário, avançado)
           - Para qual público ou nível de complexidade a explicação deve ser adaptada? (Sugira opções como para mim mesmo, para uma criança, para alguém sem conhecimento na área, nível universitário)
           - De que forma você prefere a explicação? (Sugira opções como mais resumida, bem detalhada, com muitos exemplos práticos)
        NÃO comece a falar sobre o tópico ainda.

        Em uma interação SUBSEQUENTE (quando receber as RESPOSTAS DO ALUNO):
        1. Extraia as respostas da mensagem de entrada.
        2. Consolide as respostas em um breve resumo conciso do perfil de aprendizado (Nível: [resposta], Público/Complexidade: [resposta], Estilo: [resposta]) para o TÓPICO.
        3. Sua entrega final nesta interação SUBSEQUENTE é APENAS o resumo do perfil.
        """,
        description="Agente que conversa com o estudante para entender seu perfil e necessidades.",
        tools=[]
    )
    return agente

In [ ]:
# --- Agente 2: Agente Buscador ---
# Função para criar e configurar o Agente Buscador
def criar_agente_buscador():
    print("Configurando o Agente Buscador...")
    agente = Agent(
        name="agente_buscador",
        model=MODEL_ID,
        instruction="""
        Você é um assistente de pesquisa especializado em encontrar informações relevantes e atualizadas na internet
        usando a ferramenta de busca do Google ('google_search').

        Você receberá uma MENSAGEM DE ENTRADA que conterá o TÓPICO da busca e um PERFIL DO ALUNO.
        Formato esperado da entrada:
        TÓPICO da busca: [Assunto]
        PERFIL DO ALUNO para adaptar a busca: [Perfil Sumarizado]
        ---
        [Instrução para executar a busca]

        Sua tarefa é:
        1. Extrair o TÓPICO e o PERFIL DO ALUNO da mensagem de entrada.
        2. Usar a ferramenta 'Google Search' para encontrar as informações mais importantes e relevantes sobre o TÓPICO.
        3. Adapte suas perguntas de busca ('Google Search' queries) considerando o PERFIL DO ALUNO para obter resultados adequados ao nível e complexidade desejados. Priorize resultados RECENTES e de fontes confiáveis.
        4. Sua entrega final deve ser um resumo conciso com os PONTOS-CHAVE encontrados na busca que sejam mais relevantes para o tópico e o perfil do aluno.
        5. Inclua os links (URLs) das fontes mais importantes encontradas para referência.
        NÃO comece a explicar o assunto em detalhe, apenas entregue os pontos-chave e as fontes.
        """,
        description="Agente que busca informações sobre um tópico na internet.",
        tools=[google_search] # Habilita a ferramenta Google Search
    )
    return agente

In [ ]:
# --- Agente 3: Agente Redator ---
# Função para criar e configurar o Agente Redator
def criar_agente_redator():
    print("Configurando o Agente Redator...")
    agente = Agent(
        name="agente_redator",
        model=MODEL_ID,
        instruction="""
        Você é um professor habilidoso em transformar informações complexas em aulas fáceis de entender.

        Você receberá uma MENSAGEM DE ENTRADA que conterá:
        TÓPICO da Aula: [Assunto]
        PERFIL DO ALUNO: [Perfil Sumarizado]
        RESULTADOS DA BUSCA (Pontos-Chave e Fontes):
        [Resultados da Busca]
        ---
        [Instrução para redigir]

        Sua tarefa é:
        1. Extrair o TÓPICO, o PERFIL DO ALUNO e os RESULTADOS DA BUSCA da mensagem de entrada.
        2. Criar uma explicação detalhada e envolvente sobre o TÓPICO.
        3. **É ESSENCIAL** que você adapte completamente a linguagem, a profundidade da explicação e os exemplos ao PERFIL DO ALUNO fornecido.
        4. Use os RESULTADOS DA BUSCA como a base de conteúdo para a sua explicação. NÃO invente informações que não estejam nos resultados ou sejam de conhecimento geral sobre o tópico.
        5. Estruture a aula de forma lógica e didática (introdução, desenvolvimento, conclusão).
        6. Se apropriado para o tópico e o nível do aluno, sugira visualizações (como diagramas, gráficos) ou tipos de exemplos práticos.
        7. Mencione que as fontes utilizadas estão disponíveis.

        Sua entrega final é o texto completo da aula, pronto para ser lido pelo aluno.
        """,
        description="Agente que redige o conteúdo da aula com base no tópico, perfil do aluno e resultados da busca.",
        tools=[]
    )
    return agente

In [ ]:
# --- Agente 4: Agente Review ---
# Função para criar e configurar o Agente Review
def criar_agente_review():
    print("Configurando o Agente Review...")
    agente = Agent(
        name="agente_review",
        model=MODEL_ID,
        instruction="""
        Você é um assistente de revisão e consolidação de conteúdo, especializado em criar materiais de estudo complementares.

        Você receberá uma MENSAGEM DE ENTRADA que conterá:
        TÓPICO para Revisão: [Assunto]
        PERFIL DO ALUNO: [Perfil Sumarizado]
        RESULTADOS DA BUSCA Originais (para extrair fontes e pontos-chave para resumo):
        [Resultados da Busca]
        ---
        [Instrução para gerar material de revisão]

        Sua tarefa é gerar um material de revisão focado no essencial.
        1. Extraia o TÓPICO, PERFIL DO ALUNO e RESULTADOS DA BUSCA da mensagem de entrada.
        2. Liste claramente as FONTES (URLs) que foram fornecidas nos RESULTADOS DA BUSCA originais.
        3. Crie um RESUMO em tópicos (bullet points) dos principais conceitos e fatos sobre o TÓPICO, baseando-se nos pontos-chave dos RESULTADOS DA BUSCA. Este resumo deve ser o MAIS SIMPLES e direto possível, totalmente adaptado ao NÍVEL e PÚBLICO do PERFIL DO ALUNO.

        Formate sua resposta final da seguinte maneira:
        ### Fontes Consultadas:
        [Lista de URLs]

        ### Resumo Rápido:
        [Seu resumo adaptado ao perfil do aluno em bullet points]

        Sua entrega final é este bloco de texto formatado.
        """,
        description="Agente que gera material de revisão: fontes, resumo em tópicos e sugestões visuais.",
        tools=[]
    )
    return agente

In [ ]:
# --- Agente 5: Agente Dúvidas ---
# Função para criar e configurar o Agente Dúvidas
def criar_agente_duvidas():
    print("Configurando o Agente Dúvidas...")
    agente = Agent(
        name="agente_duvidas",
        model=MODEL_ID, # Precisa de capacidade para entender perguntas, buscar e adaptar a resposta
        instruction="""
        Você é um assistente de estudo prestativo e especializado em responder perguntas sobre um tópico específico, buscando informações na internet se necessário e adaptando a resposta ao nível do usuário.

        Você receberá MENSAGENS DE ENTRADA que conterão:
        DÚVIDA DO ALUNO: [Pergunta do aluno]
        TÓPICO GERAL: [Assunto principal da sessão]
        PERFIL DO ALUNO: [Perfil Sumarizado]
        CONTEÚDO ANTERIOR (para contexto): [Texto da Aula e Resumo da sessão]
        ---
        [Instrução para responder a dúvida]

        Sua tarefa é:
        1. Extrair a DÚVIDA DO ALUNO, TÓPICO GERAL, PERFIL DO ALUNO e CONTEÚDO ANTERIOR da mensagem de entrada.
        2. Use a ferramenta 'google_search' para buscar informações que ajudem a responder à DÚVIDA DO ALUNO.
        3. Considere o CONTEÚDO fornecido como base, mas use a busca para informações adicionais ou atualizadas se necessário.
        4. Formule a resposta de forma clara e completa, **adaptando a linguagem e a profundidade EXATAMENTE ao PERFIL DO ALUNO**.
        5. Se a pergunta do aluno NÃO for sobre o TÓPICO GERAL da sessão, diga educadamente que você só pode responder perguntas relacionadas ao TÓPICO atual.
        6. Sua entrega final é APENAS a resposta adaptada à pergunta do aluno.
        """,
        description="Agente que responde dúvidas do aluno usando busca na internet.",
        tools=[google_search] # Habilita a ferramenta Google Search
    )
    return agente


In [ ]:
# --- Agente 6: Agente Salvador Local ---
# Função para criar e configurar o Agente Salvador (para salvamento LOCAL)
# Renomeado de 7 para 6 pois removemos o Questionario (5)
def criar_agente_salvador_local():
    print("Configurando o Agente Salvador Local...")
    agente = Agent(
        name="agente_salvador_local",
        model=MODEL_ID, # Um modelo simples é suficiente
        instruction="""
        Você é o assistente responsável por arquivar a sessão de estudo.

        Você receberá uma MENSAGEM DE ENTRADA que conterá TODO o conteúdo gerado durante esta sessão de estudo (a aula, o resumo, as sugestões, histórico de dúvidas).
        Formato esperado da entrada:
        CONTEÚDO COMPLETO DA SESSÃO:
        [Todo o conteúdo concatenado]
        ---
        [Instrução para perguntar sobre salvar]

        Sua única tarefa é:
        1. Reconhecer que recebeu o conteúdo da sessão.
        2. Perguntar ao aluno, de forma amigável, se ele gostaria de salvar todo este conteúdo da sessão para referência futura.
        3. Mencione que o conteúdo pode ser salvo em um arquivo aqui no ambiente local.
        NÃO salve o documento você mesmo, apenas formule e apresente a pergunta.

        Sua entrega final é APENAS a pergunta para o aluno.
        """,
        description="Agente que pergunta ao usuário se deseja salvar o histórico da sessão de estudo localmente.",
        tools=[]
    )
    return agente

In [ ]:
# @title Fluxo Principal de Execução (Salvamento Local)

# --- Início do Fluxo Principal ---

if __name__ == "__main__": # Garante que o código só roda quando o script é executado diretamente

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    print("📚 Iniciando o Sistema de Aprendizado Interativo com Agentes 🤖")
    print("-" * 60)

    # --- 1. Obter o Tópico do Usuário ---
    topico_estudo = input("❓ Olá! Por favor, digite o ASSUNTO que você gostaria de estudar hoje: ")
    print("-" * 60)

    if not topico_estudo:
        print("❌ Você esqueceu de digitar o assunto! Encerrando.")
    else:
        clear_output(wait=True) # Limpa a saída anterior no Colab para uma experiência mais fluida
        print(f"✨ Maravilha! Vamos então aprender sobre **{topico_estudo}**.")
        print("-" * 60)

        # --- 2. Interagir com o Agente Recepcionista para obter o Perfil do Aluno ---
        agente_recepcionista_obj = criar_agente_recepcionista() # Cria o objeto agente

        print("\n🤝 Conversando com o Agente Recepcionista para entender seu perfil...")
        # Primeira chamada para o Recepcionista: Faz ele apresentar e fazer as perguntas
        entrada_recepcionista_inicial = f"""
TÓPICO: {topico_estudo}
DATA: {data_de_hoje}
---
Iniciar coleta de perfil. Por favor, apresente-se e faça as perguntas para entender o perfil do aluno sobre o tópico.
"""
        resposta_do_agente = call_agent(
            agente_recepcionista_obj,
            entrada_recepcionista_inicial # Envia o tópico e data na entrada estruturada
        )
        display(to_markdown(resposta_do_agente)) # Exibe a apresentação e as perguntas do recepcionista

        # Obter as respostas do usuário para as perguntas (multiplas entradas via input)
        print("\n📝 Por favor, responda às perguntas para que eu possa preparar a aula ideal para você.")
        nivel_conhecimento = input("Seu nível de conhecimento (nenhum, básico, etc.): ")
        publico_alvo = input("Público/Complexidade (para mim, criança, sem área, etc.): ")
        estilo_preferido = input("Estilo de explicação (resumida, detalhada, etc.): ")

        # Formatar as respostas do usuário em uma única string estruturada para a SEGUNDA chamada ao Recepcionista
        respostas_do_aluno_texto = f"""
Nivel: {nivel_conhecimento}
Publico/Complexidade: {publico_alvo}
Estilo: {estilo_preferido}
"""
        clear_output(wait=True)
        print(f"✨ Vamos aprender sobre **{topico_estudo}**.")
        print("-" * 60)
        print("🤝 Respostas do perfil coletadas. Processando com Agente Recepcionista...")

        # Segunda chamada para o Recepcionista: Envia as respostas do usuário para ele processar e gerar o resumo do perfil
        entrada_recepcionista_respostas = f"""
TÓPICO: {topico_estudo} # Inclui tópico novamente para contexto, embora o agente deva lembrar
DATA: {data_de_hoje}
Respostas do Aluno:
{respostas_do_aluno_texto}
---
Processar respostas e gerar perfil sumarizado.
"""
        perfil_do_aluno_sumarizado = call_agent(
            agente_recepcionista_obj,
            entrada_recepcionista_respostas # Envia as respostas do aluno na entrada estruturada
        )
        print("\n✅ Perfil do Aluno Definido:")
        display(to_markdown(perfil_do_aluno_sumarizado)) # Exibe o perfil sumarizado
        print("-" * 60)

        # --- 3. Chamar o Agente Buscador ---
        agente_buscador_obj = criar_agente_buscador()

        # Prepara a entrada para o Buscador (usa o tópico e o perfil sumarizado na entrada estruturada)
        entrada_para_buscador = f"""
TÓPICO da busca: {topico_estudo}
PERFIL DO ALUNO para adaptar a busca: {perfil_do_aluno_sumarizado}
---
Por favor, use a ferramenta Google Search para encontrar os pontos chave e fontes relevantes.
"""
        print("\n🔍 Chamando o Agente Buscador para encontrar informações...")
        resultados_da_busca = call_agent(
            agente_buscador_obj,
            entrada_para_buscador # Envia tópico e perfil na entrada estruturada
        )
        print("\n--- 📝 Resultado da Busca (Pontos-Chave e Fontes) ---\n")
        display(to_markdown(resultados_da_busca)) # Exibe os resultados da busca
        print("-" * 60)

        # --- 4. Chamar o Agente Redator ---
        agente_redator_obj = criar_agente_redator()

        # Prepara a entrada para o Redator (usa tópico, perfil sumarizado e resultados da busca na entrada estruturada)
        entrada_para_redator = f"""
TÓPICO da Aula: {topico_estudo}
PERFIL DO ALUNO: {perfil_do_aluno_sumarizado}
RESULTADOS DA BUSCA (Pontos-Chave e Fontes):
{resultados_da_busca}
---
Por favor, redija a aula completa baseada nessas informações e adaptada ao perfil do aluno.
"""
        print("\n✍️ Chamando o Agente Redator para escrever a aula...")
        texto_final_aula = call_agent(
            agente_redator_obj,
            entrada_para_redator # Envia tópico, perfil e resultados na entrada estruturada
        )
        print("\n--- 📚 Sua Aula Está Pronta! ---\n")
        display(to_markdown(texto_final_aula)) # Exibe a aula
        print("-" * 60)

# --- 5. Agente Dúvidas Interativo ---
# Este bloco de código está fora do 'else' inicial, o que significa que ele sempre executará mesmo se o tópico não for digitado.
# Para corrigir isso e garantir que a sessão de dúvidas só comece após a aula ser gerada (dentro do fluxo principal),
# mova este bloco e os seguintes para dentro do 'else' principal.

        agente_duvidas_obj = criar_agente_duvidas()
        historico_duvidas = "" # Para armazenar o histórico das perguntas e respostas das dúvidas

        print("\n## 🤔 Tire Suas Dúvidas!")
        print("Estou pronto para responder suas perguntas sobre o tópico. Pergunte o que quiser!")
        print("Para encerrar a sessão de dúvidas e salvar o histórico, digite 'sair'.")
        print("-" * 60)

        # Loop de Dúvidas e Respostas
        continuar_duvidas = True
        while continuar_duvidas:
            # Captura a dúvida do aluno ou comando de saída
            duvida_aluno = input("Sua dúvida ou 'sair': ")

            # Verifica se o aluno quer sair
            if duvida_aluno.strip().lower() == 'sair':
                print("\nEncerrando a sessão de dúvidas.")
                continuar_duvidas = False # Sai do loop
                break # Garante saída imediata

            # Se o aluno não quer sair, prepara a entrada para o Agente Dúvidas
            # É necessário gerar o material_revisao ANTES de usá-lo no contexto das dúvidas.
            # Adicionar a chamada para o Agente Review aqui, ANTES do loop de dúvidas.

            # --- Chamar o Agente Review ANTES do loop de dúvidas ---
            agente_review_obj = criar_agente_review() # Cria o objeto agente

            # Prepara a entrada para o Review
            entrada_para_review = f"""
TÓPICO para Revisão: {topico_estudo}
PERFIL DO ALUNO: {perfil_do_aluno_sumarizado}
RESULTADOS DA BUSCA Originais (para extrair fontes e pontos-chave para resumo):
{resultados_da_busca}
---
Por favor, gere o material de revisão.
"""
            print("\n✨ Gerando material de revisão...")
            material_revisao = call_agent(
                agente_review_obj,
                entrada_para_review
            )
            print("\n--- 📖 Material de Revisão ---\n")
            display(to_markdown(material_revisao))
            print("-" * 60)
            # --- Fim da chamada ao Agente Review ---

            # Se o aluno não quer sair, prepara a entrada para o Agente Dúvidas
            entrada_para_duvidas = f"""
DÚVIDA DO ALUNO: {duvida_aluno}
TÓPICO GERAL: {topico_estudo}
PERFIL DO ALUNO: {perfil_do_aluno_sumarizado}
CONTEÚDO ANTERIOR (para contexto):
{texto_final_aula}
{material_revisao}
---
Por favor, responda a dúvida do aluno utilizando busca na internet e adaptando ao perfil.
"""
            print("\nBuscando a resposta para sua dúvida...")
            # Chama o Agente Dúvidas
            resposta_do_agente_duvidas = call_agent(
                agente_duvidas_obj,
                entrada_para_duvidas # Envia a dúvida e o contexto na entrada estruturada
            )

            # Adiciona a pergunta e resposta ao histórico ANTES de exibir
            historico_duvidas += f"Aluno Dúvida: {duvida_aluno}\n"
            historico_duvidas += f"Professor Dúvidas: {resposta_do_agente_duvidas}\n"
            historico_duvidas += "---\n" # Separador no histórico

            display(to_markdown(resposta_do_agente_duvidas)) # Exibe a resposta do agente
            print("---\n") # Separador visual no console


        # --- Fim do Loop de Dúvidas ---
        print("\n✅ Sessão de Dúvidas Encerrada.")
        print("-" * 60)


        # --- 6. Lógica para Salvar em Arquivo Local ---
        # O Agente Salvador Local é usado para FAZER a pergunta sobre salvar.
        # A lógica de salvamento é executada no script principal.

        agente_salvador_local_obj = criar_agente_salvador_local()

        # Prepara todo o conteúdo da sessão para possivelmente salvar localmente
        # Inclui o histórico das dúvidas
        conteudo_completo_sessao = f"""
# Relatório da Sessão de Estudo: {topico_estudo}

**Data:** {data_de_hoje}
**Perfil do Aluno:** {perfil_do_aluno_sumarizado}

---

## Conteúdo da Aula

{texto_final_aula}

---

## Resumo e Materiais Adicionais

{material_revisao}

---

## Histórico da Sessão de Dúvidas

{historico_duvidas.strip() if historico_duvidas.strip() else "Nenhum histórico de dúvidas registrado."}

---

""" # Concatena todo o conteúdo relevante, incluindo o histórico das dúvidas

        print("\n💾 Finalizando a sessão de estudo e preparando para salvar...")
        print("Chamando o Agente Salvador para perguntar sobre salvar...")

        # Chama o Agente Salvador Local para fazer a pergunta.
        # Passamos uma amostra do conteúdo como contexto, embora ele só precise da instrução para perguntar.
        entrada_para_salvador_local = f"""
CONTEÚDO COMPLETO DA SESSÃO (amostra):
{conteudo_completo_sessao[:1500]} # Amostra do conteúdo
---
Por favor, faça a pergunta sobre salvar o conteúdo completo desta sessão em um arquivo local.
"""

        pergunta_do_salvador = call_agent(
            agente_salvador_local_obj,
            entrada_para_salvador_local # Passa o contexto para a pergunta
        )
        display(to_markdown(pergunta_do_salvador)) # Exibe a pergunta sobre salvar localmente

        # --- Lógica para Capturar a Resposta do Usuário e EXECUTAR A AÇÃO DE SALVAR (FORA DO AGENTE) ---
        # Pergunta diretamente ao usuário no script principal se deseja salvar
        resposta_salvar = input("\nPor favor, digite 'Sim' para salvar o arquivo local ou 'Não' para pular: ").strip().lower()

        if resposta_salvar == 'sim':
            print("\nOk! Preparando para salvar o conteúdo da sessão em um arquivo local...")

            # --- IMPLEMENTAÇÃO SIMPLES: Salvar em um arquivo .md no ambiente Colab ---
            # Define o nome do arquivo
            nome_arquivo_local = f"Sessao_Estudo_{topico_estudo.replace(' ', '_').replace('/', '-')}_{date.today().strftime('%Y%m%d')}.md"
            try:
                 # Abre o arquivo em modo de escrita ('w') com encoding UTF-8
                 with open(nome_arquivo_local, "w", encoding="utf-8") as f:
                     f.write(conteudo_completo_sessao) # Escreve TODO o conteúdo concatenado
                 print(f"✅ Conteúdo salvo com sucesso em '{nome_arquivo_local}' no seu ambiente Colab.")
                 print("Você pode encontrar e baixar este arquivo na barra lateral do Colab (ícone de pasta -> 📁).")
            except Exception as e:
                 print(f"❌ Ocorreu um erro ao tentar salvar o arquivo local: {e}")

        else: # Este else deve estar alinhado com o if resposta_salvar == 'sim':
            print("\nOk, o conteúdo da sessão de estudo não será salvo neste momento.")

        print("\n--- 👋 Fim da Sessão de Estudo Interativo! 👋 ---")
        print("-" * 60)

# --- Fim do Bloco Principal ---

📚 Iniciando o Sistema de Aprendizado Interativo com Agentes 🤖
------------------------------------------------------------
